# Exercise: SIMD Data Dependency

Consider the following loop involving four vectors `a`,`b`,`c`, and `d`:

In [1]:
const LOOP_ITERATIONS = 8192
const N = LOOP_ITERATIONS + 2

function loop_naive!(a, b, c, d)
    @inbounds for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
        b[i+2] = c[i] + d[i]
    end
end

a = rand(Float32, N)
b = rand(Float32, N)
c = rand(Float32, N)
d = rand(Float32, N)

loop_naive!(a,b,c,d)

This loop is hard to auto-vectorize because it has a **data-dependency**: we're reading and writing elements of the vector `b`.

**Task 1**: Check the native code produced for `loop_naive!(a,b,c,d)` and convince yourself that the Julia compiler hasn't vectorized this code. (There shouldn't be any usage of `ymm` or `zmm` registers etc.)

In [2]:
@code_native loop_naive!(a,b,c,d)

	.text
	.file	"loop_naive!"
	.globl	"japi1_loop_naive!_769"         # -- Begin function japi1_loop_naive!_769
	.p2align	4, 0x90
	.type	"japi1_loop_naive!_769",@function
"japi1_loop_naive!_769":                # @"japi1_loop_naive!_769"
; ┌ @ In[1]:4 within `loop_naive!`
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	mov	qword ptr [rbp - 56], rsi
	mov	rax, qword ptr [rsi]
	mov	rcx, qword ptr [rsi + 8]
	mov	rdx, qword ptr [rsi + 16]
	mov	rsi, qword ptr [rsi + 24]
	mov	rax, qword ptr [rax]
	mov	rcx, qword ptr [rcx]
	mov	rdx, qword ptr [rdx]
	mov	rsi, qword ptr [rsi]
; │ @ In[1]:5 within `loop_naive!`
	lea	rdi, [rax + 32768]
	lea	rbx, [rcx + 32776]
	lea	r8, [rdx + 32768]
	lea	r10, [rsi + 32768]
	cmp	rax, rbx
	setb	r13b
	cmp	rcx, rdi
	setb	r14b
	cmp	rax, r8
	setb	r11b
	cmp	rdx, rdi
	setb	r12b
	cmp	rax, r10
	setb	r9b
	cmp	rsi, rdi
	setb	r15b
	cmp	rcx, r8
	setb	r8b
	cmp	rdx, rbx
	setb	byte ptr [rbp - 41]             # 


**Task 2**: Implement the same loop in `loop_naive_simd!` and try to force SIMD-vectorization with the corresponding performance macro. (You shall keep the `@inbounds` as well.)

In [3]:
function loop_naive_simd!(a, b, c, d)
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
        b[i+2] = c[i] + d[i]
    end
end

loop_naive_simd! (generic function with 1 method)

**Task 3**: Check the native code of `loop_naive_simd!`. Has the code improved? The learning here is that just putting `@simd` in front of a loop and hoping for the best isn't a particularly good strategy 😉

In [4]:
@code_native loop_naive_simd!(a,b,c,d)

	.text
	.file	"loop_naive_simd!"
	.globl	"japi1_loop_naive_simd!_878"    # -- Begin function japi1_loop_naive_simd!_878
	.p2align	4, 0x90
	.type	"japi1_loop_naive_simd!_878",@function
"japi1_loop_naive_simd!_878":           # @"japi1_loop_naive_simd!_878"
; ┌ @ In[3]:1 within `loop_naive_simd!`
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	mov	qword ptr [rbp - 56], rsi
	mov	rax, qword ptr [rsi]
	mov	rcx, qword ptr [rsi + 8]
	mov	rdx, qword ptr [rsi + 16]
	mov	rsi, qword ptr [rsi + 24]
	mov	rax, qword ptr [rax]
	mov	rcx, qword ptr [rcx]
	mov	rdx, qword ptr [rdx]
	mov	rsi, qword ptr [rsi]
; │ @ In[3]:2 within `loop_naive_simd!`
; │┌ @ simdloop.jl:75 within `macro expansion`
	lea	rdi, [rax + 32768]
	lea	rbx, [rcx + 32776]
	lea	r8, [rdx + 32768]
	lea	r10, [rsi + 32768]
	cmp	rax, rbx
	setb	r13b
	cmp	rcx, rdi
	setb	r14b
	cmp	rax, r8
	setb	r11b
	cmp	rdx, rdi
	setb	r12b
	cmp	rax, r10
	setb	r9b
	cmp	rsi, rdi
	setb	r15b


**Task 4**: Benchmark and compare the variants. What do you observe?


In [7]:
using BenchmarkTools
@btime loop_naive!($a,$b,$c,$d);
@btime loop_naive_simd!($a,$b,$c,$d);

  6.098 μs (0 allocations: 0 bytes)
  6.098 μs (0 allocations: 0 bytes)



**Task 5**: Take a closer look at the loop. Can you "resolve" the data-dependency issue by splitting up the loop into two separate loops? Implement this improved version in the functions below. Use `@simd` for the loops in the second function. (Again, keep `@inbounds` for all loops in both functions.)

In [19]:
function loop_opt!(a, b, c, d)
    bcpy = copy(b)
    @inbounds for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + bcpy[i]
        b[i+2] = c[i] + d[i]
    end
end

function loop_opt_simd!(a, b, c, d)
    bcpy = deepcopy(b)
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + bcpy[i]
        b[i+2] = c[i] + d[i]
    end
end

loop_opt_simd! (generic function with 1 method)

**Task 6**: Benchmark those new variants as well.
  * How do they compare to each other?
  * Did the SIMD performance macro help? (Hint: It shouldn't.)
  * How does the performance compare to the unoptimized variants above?

In [20]:
@btime loop_opt!($a,$b,$c,$d);
@btime loop_opt_simd!($a,$b,$c,$d);

  5.503 μs (2 allocations: 32.11 KiB)
  5.635 μs (4 allocations: 32.44 KiB)



**Task 7**: Check the native code of e.g. `loop_opt_simd!`. Did it vectorize properly? (Look e.g. for `ymm` and `zmm` registers as well as a block of `vaddps` instructions. Note though, that this is system-dependent.)

In [21]:
@code_native loop_opt_simd!(a,b,c,d)

	.text
	.file	"loop_opt_simd!"
	.globl	"japi1_loop_opt_simd!_1481"     # -- Begin function japi1_loop_opt_simd!_1481
	.p2align	4, 0x90
	.type	"japi1_loop_opt_simd!_1481",@function
"japi1_loop_opt_simd!_1481":            # @"japi1_loop_opt_simd!_1481"
; ┌ @ In[19]:9 within `loop_opt_simd!`
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	sub	rsp, 56
	vxorps	xmm0, xmm0, xmm0
	vmovaps	xmmword ptr [rbp - 80], xmm0
	mov	qword ptr [rbp - 64], 0
	mov	qword ptr [rbp - 96], rsi
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r12, qword ptr [rax - 8]
	mov	qword ptr [rbp - 80], 4
	mov	rax, qword ptr [r12]
	mov	qword ptr [rbp - 72], rax
	lea	rax, [rbp - 80]
	mov	qword ptr [r12], rax
	mov	rax, qword ptr [rsi]
	mov	qword ptr [rbp - 56], rax       # 8-byte Spill
	mov	r14, qword ptr [rsi + 8]
	mov	r13, qword ptr [rsi + 16]
	mov	r15, qword ptr [rsi + 24]
	movabs	rax, 140292601333936
; │ @ In[19]:10 within `loop_opt_simd!`
; │┌ @ de